# Proyecto Final IH: Recomendador de Codewars|Github

In [ ]:
# https://www.codewars.com/users/leaderboard

In [11]:
# imports 

import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
from IPython.display import display

pd.options.display.max_columns = None

## Fase 1: Obtención de usuarios de forma iterativa

### Leaderboard
Top 500 de codewars

In [12]:
# URL

url = 'https://www.codewars.com/users/leaderboard'
html = requests.get(url).content
soup = BeautifulSoup(html, "lxml")

In [13]:
# usuarios de leaderboard
def get_top_500(url='https://www.codewars.com/users/leaderboard'): 
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")
    return {e.text for e in soup.select('tr a')}

# users = '\n'.join(get_top_500())
# set_top500
# users
# get_top_500()

In [14]:
# Salvamos los nombres en un documento
def save_set_users(users, filename='../output/codewars-users.txt'): 
    ''' Save set/list of users to file '''
    with open(filename, 'w') as f: 
        f.write('\n'.join(users)) 
# save_set_users({'hola', 'adios'})

In [15]:
def save_string_users(users, filename='../output/codewars-users.txt'): 
    ''' Save string of users to file '''
    with open(filename, 'w') as f: 
        f.write(users) 
# save_set_users(users)

In [16]:
def add_set_users(users, filename='../output/codewars-users.txt'): 
    ''' Append to file '''
    with open(filename, 'a') as f: 
        f.write(users) 
# add_set_users(users)

In [17]:
def load_set_users(filename='../output/codewars-users.txt'): 
    ''' Read from file '''
    with open(filename, 'r') as f: 
        users = f.readlines()
    return {user.strip() for user in users}
# len(load_set_users())

### Get Social

In [18]:
# get users in clan

# https://www.codewars.com/users/albertogcmr/following
# https://www.codewars.com/users/albertogcmr/followers
# https://www.codewars.com/users/albertogcmr/allies

def get_social(user): 
    res = set()
    for link in ['following', 'followers', 'allies']: 
        url = 'https://www.codewars.com/users/{}/{}'.format(user, link)
        html = requests.get(url).content

        soup = BeautifulSoup(html, "lxml")
        res = res.union({e.text for e in soup.select('table a')})
    return res

# get_social('albertogcmr')

In [19]:
### Get stats from user
'''
def get_all_stats(user): 
    url = 'https://www.codewars.com/users/{}'.format(user)
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")

    x = {}
    x['username'] = user
    for s in soup.select('.stat-box div'): 
        if s.text.split(':')[0] != 'Profiles': 
            x[s.text.split(':')[0].lower()] = s.text.split(':')[1]
        else: 
            try: 
                for e in s.find_all('a', href=True): 
                    if 'github' in e['href']: 
                        x['github'] = e['href']
                    if 'linkedin' in e['href']: 
                        x['linkedin'] = e['href']
            except: 
                x['github'] = ''
                x['linkedin'] = ''
    return x

get_all_stats('albertogcmr')

'''
pass

### Iteración para ampliar número de usuarios

A partir de la semilla inicial de 500 obtenemos casi 4000 usuarios de codewars

In [ ]:
top500 = get_top_500()
total = set().union(top500)
users = total

In [ ]:
print(len(total))

for i, user in enumerate(users): 
    total = total.union(get_social(user))
    print(i, '\t', len(total))

print(len(total))

In [ ]:
# Me añado a mí mismo

total = total.union({'albertogcmr'})
for user in get_social('albertogcmr'): 
    total = total.union(get_social(user))
len(total)

### Persistencia de datos en un txt

In [20]:
# save_set_users(users=total, filename='../output/no-borrar-usuarioscodewars-2.txt')
total = load_set_users('../output/no-borrar-usuarioscodewars-2.txt')

In [21]:
len(total)

3953

## Fase 2: Obtención de datos de usuario
### 2.1 API codewars

In [22]:
# GET user

def get_user_api(user='albertogcmr'): 
    ''' Obtenemos el json de la API de codewars '''
    url = 'https://www.codewars.com/api/v1/users/{}'.format(user)
    response = requests.get(url)
    return response.json()

get_user_api('albertogcmr')

{'username': 'albertogcmr',
 'name': 'Alberto García cobo',
 'honor': 727,
 'clan': 'Ironhack',
 'leaderboardPosition': 12893,
 'skills': ['java',
  'c',
  'python',
  'jquery',
  'javascript',
  'crm',
  'erp',
  'itil'],
 'ranks': {'overall': {'rank': -4,
   'name': '4 kyu',
   'color': 'blue',
   'score': 888},
  'languages': {'python': {'rank': -4,
    'name': '4 kyu',
    'color': 'blue',
    'score': 857},
   'sql': {'rank': -7, 'name': '7 kyu', 'color': 'white', 'score': 24},
   'c': {'rank': -8, 'name': '8 kyu', 'color': 'white', 'score': 10}}},
 'codeChallenges': {'totalAuthored': 0, 'totalCompleted': 89}}

In [23]:
def get_value_from_json(user_json, value='username'): 
    ''' 
    value en nivel 1 del json: 
    possible values = ['username', 'honor', 'clan', 'leaderboardPosition', 'skills']
    '''
    return user_json[value]

def get_score_language(user_json, language): 
    try: 
        score = user_json['ranks']['languages'][language]['score']
    except: 
        score = 0
    return score


def get_scores(user_json): 
    res = {}
    for lang in get_languages(): 
        res[lang] = get_score_language(user_json, lang)
    return res

# get_score_language(user_json=alberto, language='sca')
# get_value_from_json(alberto, 'skills')
# get_scores(alberto)

### 2.2 Web Scrapping codewars

Get stats from user

In [24]:
def get_all_stats(user): 
    url = 'https://www.codewars.com/users/{}'.format(user)
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")

    x = {}
    x['username'] = user
    for s in soup.select('.stat-box div'): 
        if s.text.split(':')[0] != 'Profiles': 
            x[s.text.split(':')[0].lower()] = s.text.split(':')[1]
        else: 
            try: 
                for e in s.find_all('a', href=True): 
                    if 'github' in e['href']: 
                        x['github'] = e['href']
                    if 'linkedin' in e['href']: 
                        x['linkedin'] = e['href']
            except: 
                x['github'] = ''
                x['linkedin'] = ''
    return x

get_all_stats('albertogcmr')

{'username': 'albertogcmr',
 'name': 'Alberto García cobo',
 'clan': 'Ironhack',
 'skills': 'java, c, python, jquery, javascript, crm, erp, itil',
 'member since': 'Oct 2018',
 'last seen': 'Dec 2018',
 'github': 'https://github.com/albertogcmr',
 'linkedin': 'https://www.linkedin.com/in/albertogarciacobo/',
 'following': '593',
 'followers': '591',
 'allies': '591',
 'rank': '4 kyu',
 'honor': '727',
 'leaderboard position': '#12,893',
 'honor percentile': '98th',
 'total completed kata': '89',
 'total languages trained': '3',
 'highest trained': 'Python (4 kyu)',
 'most recent': 'Python',
 'comments': '0 (0 replies)',
 'collections': '0',
 'kumite': '0',
 'translations': '0 (0 approved)'}

In [25]:
def get_languages(url='https://www.codewars.com/kata/latest/my-languages', ignore={'all', 'my languages'}): 
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")
    return {e.text.lower() for e in soup.select('#language_filter option')}.difference(ignore) 

get_languages()

{'bf (beta)',
 'c',
 'c#',
 'c++',
 'clojure',
 'coffeescript',
 'crystal',
 'dart',
 'elixir',
 'elm (beta)',
 'erlang (beta)',
 'f#',
 'fortran (beta)',
 'go',
 'groovy (beta)',
 'haskell',
 'java',
 'javascript',
 'julia (beta)',
 'kotlin (beta)',
 'lua (beta)',
 'nasm (beta)',
 'nim (beta)',
 'objective-c (beta)',
 'ocaml (beta)',
 'php',
 'powershell (beta)',
 'purescript (beta)',
 'python',
 'r (beta)',
 'ruby',
 'rust',
 'scala (beta)',
 'shell',
 'solidity (beta)',
 'sql',
 'swift',
 'typescript'}

In [26]:
def get_row(user): 
    ''' 
    Crea un diccionario con los datos recopilados que se puede
    añadir como fila en nuestro dataframe de pandas
    '''
    user_json = get_user_api(user)
    res = get_all_stats(user)
    res.update(get_scores(user_json))
    return res

# ivan = get_row('ijcernicharo')
# alberto = get_row('albertogcmr')

In [27]:
def create_row(data, new_user): 
    row = get_row(new_user)
    return data.append(other=row, ignore_index=True)

### 2.3 Creación de DataFrame

In [28]:
df_users = pd.DataFrame()

In [29]:
# Por cada uno de los usuarios recopilados, creamos un registro en df_users
total_500 = [u for u in total][:]
fallos = []

# iniciar la medición de tiempo
for i, user in enumerate(total_500): 
    try: 
        df_users = create_row(data=df_users, new_user=user)
        print(i)
    except: 
        print('Fallo:', user)
        fallos.append(user)

# terminar la medición de tiempo        
df_users.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
Fa

,allies,bf (beta),c,c#,c++,clan,clojure,coffeescript,collections,comments,crystal,dart,elixir,elm (beta),erlang (beta),f#,followers,following,fortran (beta),go,groovy (beta),haskell,highest trained,honor,honor percentile,java,javascript,julia (beta),kotlin (beta),kumite,last seen,leaderboard position,lua (beta),member since,most recent,name,nasm (beta),nim (beta),objective-c (beta),ocaml (beta),php,powershell (beta),purescript (beta),python,r (beta),rank,ruby,rust,scala (beta),shell,solidity (beta),sql,swift,total completed kata,total languages trained,translations,typescript,username,algorithms,avg. rank,avg. satisfaction rating,created,data structures,fundamentals,github,graphs,logic,skills,total collected,total completions,total stars,contributed kata,kata approvals,puzzles,linkedin,1 kyu,2 kyu,3 kyu,4 kyu,5 kyu,6 kyu,7 kyu,8 kyu,authored translations,best practice,best practice solutions,clever,clever solutions,completed kata,completed on 1st attempt,current streak,date,ended on,first completed,kata attempts,last completed,most number of days,arrays,basic language features,bugs,data types,data,io,streams,closures,games,dates/time,functions,object-oriented programming,refactoring,lists,mathematics,numbers,programming paradigms,declarative programming,functional programming,strings,graphics,classes,control flow,security,binary,algebra,advanced language features,objects,esoteric languages,interpreters,rules,formats,prototypes,sparse arrays,bits,bitwise operators,performance,permutations,sorting,trees,big integers,sequences,arithmetic,filtering,loops,statistics,hashes,formatting,authored kata,data conversion,map/reduce,hacking holidays,maps,utilities,encoding,inheritance,polymorphism,geometry,databases,information systems,regular expressions,theoretical computer science,cryptography,encryption,parsing,decryption,lambdas,search,integers,es2015,methods,variables,higher-order functions,nodejs,queues,json
0,0,0.0,55.0,0.0,0.0,Unknown,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.0,0.0,0.0,1573.0,Haskell (4 kyu),720,97th,0.0,0.0,0.0,0.0,0,Jul 2018,"#13,112",0.0,Aug 2015,C,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4 kyu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28,2,0 (0 approved),0.0,uie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.0,87.0,0.0,0.0,Informatic in Engineering | Faculty of Enginee...,0.0,0.0,1,7 (1 replies),0.0,0.0,0.0,0.0,0.0,0.0,2,23,0.0,0.0,0.0,0.0,JavaScript (3 kyu),"4,727",99th,169.0,3632.0,0.0,0.0,3 (2 Started),Dec 2018,#589,0.0,Dec 2017,JavaScript,NIKOLA NIKOLIC,0.0,0.0,0.0,0.0,1018.0,0.0,0.0,0.0,0.0,3 kyu,43.0,28.0,0.0,0.0,0.0,22.0,0.0,734,8,4 (1 approved),0.0,CRYPTON,1,7 kyu,30%,1 (0 in beta),1,3,https://github.com/CRYPPTON,1,1,javascript | php,3,347,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0.0,0.0,0.0,0.0,hi9,0.0,0.0,0,1 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,0,3,0.0,0.0,0.0,0.0,JavaScript (6 kyu),223,90th,0.0,210.0,0.0,0.0,2 (2 Started),May 2018,"#64,759",0.0,Mar 2016,JavaScript,Marcus Hitchins,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6 kyu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21,1,0 (0 approved),0.0,marcus7777,NaN,NaN,NaN,NaN,NaN,NaN,https://github.com/marcus7777,NaN,NaN,"cycling, dressmaking (still learning), polymer...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [34]:
df_users = create_row(data=df_users, new_user='ijcernicharo')
df_users = create_row(data=df_users, new_user='pablobarrio')
df_users.head()

,allies,bf (beta),c,c#,c++,clan,clojure,coffeescript,collections,comments,crystal,dart,elixir,elm (beta),erlang (beta),f#,followers,following,fortran (beta),go,groovy (beta),haskell,highest trained,honor,honor percentile,java,javascript,julia (beta),kotlin (beta),kumite,last seen,leaderboard position,lua (beta),member since,most recent,name,nasm (beta),nim (beta),objective-c (beta),ocaml (beta),php,powershell (beta),purescript (beta),python,r (beta),rank,ruby,rust,scala (beta),shell,solidity (beta),sql,swift,total completed kata,total languages trained,translations,typescript,username,algorithms,avg. rank,avg. satisfaction rating,created,data structures,fundamentals,github,graphs,logic,skills,total collected,total completions,total stars,contributed kata,kata approvals,puzzles,linkedin,1 kyu,2 kyu,3 kyu,4 kyu,5 kyu,6 kyu,7 kyu,8 kyu,authored translations,best practice,best practice solutions,clever,clever solutions,completed kata,completed on 1st attempt,current streak,date,ended on,first completed,kata attempts,last completed,most number of days,arrays,basic language features,bugs,data types,data,io,streams,closures,games,dates/time,functions,object-oriented programming,refactoring,lists,mathematics,numbers,programming paradigms,declarative programming,functional programming,strings,graphics,classes,control flow,security,binary,algebra,advanced language features,objects,esoteric languages,interpreters,rules,formats,prototypes,sparse arrays,bits,bitwise operators,performance,permutations,sorting,trees,big integers,sequences,arithmetic,filtering,loops,statistics,hashes,formatting,authored kata,data conversion,map/reduce,hacking holidays,maps,utilities,encoding,inheritance,polymorphism,geometry,databases,information systems,regular expressions,theoretical computer science,cryptography,encryption,parsing,decryption,lambdas,search,integers,es2015,methods,variables,higher-order functions,nodejs,queues,json
0,0,0.0,55.0,0.0,0.0,Unknown,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.0,0.0,0.0,1573.0,Haskell (4 kyu),720,97th,0.0,0.0,0.0,0.0,0,Jul 2018,"#13,112",0.0,Aug 2015,C,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4 kyu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28,2,0 (0 approved),0.0,uie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.0,87.0,0.0,0.0,Informatic in Engineering | Faculty of Enginee...,0.0,0.0,1,7 (1 replies),0.0,0.0,0.0,0.0,0.0,0.0,2,23,0.0,0.0,0.0,0.0,JavaScript (3 kyu),"4,727",99th,169.0,3632.0,0.0,0.0,3 (2 Started),Dec 2018,#589,0.0,Dec 2017,JavaScript,NIKOLA NIKOLIC,0.0,0.0,0.0,0.0,1018.0,0.0,0.0,0.0,0.0,3 kyu,43.0,28.0,0.0,0.0,0.0,22.0,0.0,734,8,4 (1 approved),0.0,CRYPTON,1,7 kyu,30%,1 (0 in beta),1,3,https://github.com/CRYPPTON,1,1,javascript | php,3,347,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0.0,0.0,0.0,0.0,hi9,0.0,0.0,0,1 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,0,3,0.0,0.0,0.0,0.0,JavaScript (6 kyu),223,90th,0.0,210.0,0.0,0.0,2 (2 Started),May 2018,"#64,759",0.0,Mar 2016,JavaScript,Marcus Hitchins,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6 kyu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21,1,0 (0 approved),0.0,marcus7777,NaN,NaN,NaN,NaN,NaN,NaN,https://github.com/marcus7777,NaN,NaN,"cycling, dressmaking (still learning), polymer...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [35]:
df_users.shape

(997, 173)

In [36]:
df_users.to_csv('../output/df-173columnas.txt')



In [ ]:
'linkedin' in df_users.columns

In [ ]:
df_users = create_row(data=df_users, new_user='albertogcmr')
df_users.head(7)

In [ ]:
'linkedin' in df_users.columns

In [ ]:
df_users.to_csv('../output/dataframe.csv')
df = pd.read_csv('../output/dataframe.csv')
df.head()

In [ ]:
df.games.value_counts()


### 2.4 Limpieza de datos

Tenemos más de 100 columnas de las que sólo queremos las concernientes a sus datos de programación

In [ ]:
# Nos vamos a quedar con las siguientes columnas
get_languages()
allies, clan, comments, followers, following, github, honor, last seen, 
leaderboard position # eliminar el #
member since, total completed kata, total languages trained, translations, username, avg. satisfaction rating, 
contributed kata, created, data structures, data types, fundamentals, graphs, kata approvals, 
total collected, total completions, total stars, skills, linkedin


## Fase 3: Persistencia de datos

In [ ]:
Guarda